In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_probability as tfp
import sklearn.metrics as metrics
import os
import numpy as np
import pandas as pd
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement = True  
sess = tf.compat.v1.Session(config=config)
set_session(sess)
import tensorflow.compat.v1 as tf
tf.compat.v1.disable_eager_execution()
import matplotlib.pyplot as plt

Using TensorFlow backend.


Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce GTX 960M, pci bus id: 0000:01:00.0, compute capability: 5.0



In [2]:
train_data_dir = 'chest_xray/chest_xray/train'
validation_data_dir = 'chest_xray/chest_xray/val'
test_data_dir = 'chest_xray/chest_xray/test'

train_pneumonia_dir = 'chest_xray/chest_xray/train/NORMAL'
train_normal_dir = 'chest_xray/chest_xray/train/PNEUMONIA'

test_pneumonia_dir = 'chest_xray/chest_xray/test/NORMAL'
test_normal_dir = 'chest_xray/chest_xray/test/PNEUMONIA'

validation_pneumonia_dir = 'chest_xray/chest_xray/val/NORMAL'
validation_normal_dir = 'chest_xray/chest_xray/val/PNEUMONIA'


In [3]:
nb_train_normal_samples = len([f for f in os.listdir(train_normal_dir)if os.path.isfile(os.path.join(train_normal_dir, f))])
nb_train_pneumonia_samples = len([f for f in os.listdir(train_pneumonia_dir)if os.path.isfile(os.path.join(train_pneumonia_dir, f))])

nb_test_normal_samples = len([f for f in os.listdir(test_normal_dir)if os.path.isfile(os.path.join(test_normal_dir, f))])
nb_test_pneumonia_samples = len([f for f in os.listdir(test_pneumonia_dir)if os.path.isfile(os.path.join(test_pneumonia_dir, f))])

nb_validation_normal_samples = len([f for f in os.listdir(validation_normal_dir)if os.path.isfile(os.path.join(validation_normal_dir, f))])
nb_validation_pneumonia_samples = len([f for f in os.listdir(validation_pneumonia_dir)if os.path.isfile(os.path.join(validation_pneumonia_dir, f))])

nb_train_samples = nb_train_normal_samples + nb_train_pneumonia_samples 
nb_test_samples = nb_test_normal_samples + nb_test_pneumonia_samples 
nb_validation_samples = nb_validation_normal_samples + nb_validation_pneumonia_samples 

epochs = 20
batch_size = 16

print("Number of training samples:" ,nb_train_samples, "\n")
print("Number of validation samples:", nb_validation_samples, "\n")
print("Number of testing samples:", nb_test_samples, "\n")

Number of training samples: 5216 

Number of validation samples: 16 

Number of testing samples: 624 



In [4]:
img_width, img_height= 150, 150

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)


In [5]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(1))
model.add(Activation('sigmoid'))
print(model.summary())

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
activation (Activation)      (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________

In [6]:
modelBayes = tf.keras.Sequential([
    tf.keras.Input(shape=input_shape),
    tfp.layers.Convolution2DReparameterization(32, (3,3),  padding="SAME", activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D(pool_size=[2, 2],  padding="SAME"),
    
    tfp.layers.Convolution2DReparameterization(32, (3,3),  padding="SAME", activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D(pool_size=[2, 2],  padding="SAME"),
    
    tfp.layers.Convolution2DReparameterization(64, (3,3),  padding="SAME", activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D(pool_size=[2, 2],  padding="SAME"),
    
    tf.keras.layers.Flatten(),
    tfp.layers.DenseFlipout(128, activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.5),
    tfp.layers.DenseFlipout(64, activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1)
])
print(modelBayes.summary())

Instructions for updating:
Please use `layer.add_weight` method instead.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_reparameterization (C (None, 150, 150, 32)      1760      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_reparameterization_1  (None, 75, 75, 32)        18464     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 38, 38, 32)        0         
_________________________________________________________________
conv2d_reparameterization_2  (None, 38, 38, 64)        36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 19, 19, 64)        0         
_______________________________________________

In [7]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

modelBayes.compile(loss='binary_crossentropy',
                   optimizer='rmsprop',
                   metrics=['accuracy'])


In [8]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')


Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [9]:
checkpointer = ModelCheckpoint(filepath = 'CNN_binary.hdf5', verbose = 1, save_best_only = True)

model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks = [checkpointer],
    validation_steps=validation_generator.samples // batch_size)

model.save_weights('CNN_binary_last.hdf5')


Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/20
  1/326 [..............................] - ETA: 4:33 - loss: 0.4313 - accuracy: 0.8125
Epoch 00001: val_loss improved from inf to 0.43127, saving model to CNN_binary.hdf5
326/326 [==============================] - 144s 441ms/step - loss: 0.4708 - accuracy: 0.8033 - val_loss: 0.4313 - val_accuracy: 0.8125
Epoch 2/20
  1/326 [..............................] - ETA: 1:34 - loss: 0.4903 - accuracy: 0.8750
Epoch 00002: val_loss did not improve from 0.43127
326/326 [==============================] - 135s 414ms/step - loss: 0.2874 - accuracy: 0.8871 - val_loss: 0.4903 - val_accuracy: 0.8750
Epoch 3/20
  1/326 [..............................] - ETA: 1:23 - loss: 0.7692 - accuracy: 0.6875
Epoch 00003: val_loss did not improve from 0.43127
326/326 [==============================] - 108s 333ms/step - loss: 0.2503 - accuracy: 0.9133 - val_loss: 0.7692 - val_accuracy: 0.6875
Epoch 4/20
  1/326 [...................

In [10]:
checkpointer = ModelCheckpoint(filepath = 'Bayesyian_CNN_binary.hdf5', verbose = 1, save_best_only = True)

modelBayes.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks = [checkpointer],
    validation_steps=validation_generator.samples // batch_size)

modelBayes.save_weights('Bayesyian_CNN_binary_Final.hdf5')


Epoch 1/20
  1/326 [..............................] - ETA: 6:28 - loss: 6590475.5000 - accuracy: 0.5000
Epoch 00001: val_loss improved from inf to 6590475.50000, saving model to Bayesyian_CNN_binary.hdf5
326/326 [==============================] - 113s 347ms/step - loss: 7066250.3988 - accuracy: 0.7178 - val_loss: 6590475.5000 - val_accuracy: 0.5000
Epoch 2/20
  1/326 [..............................] - ETA: 1:25 - loss: 5658572.5000 - accuracy: 0.5000
Epoch 00002: val_loss improved from 6590475.50000 to 5658572.50000, saving model to Bayesyian_CNN_binary.hdf5
326/326 [==============================] - 111s 340ms/step - loss: 6124667.5890 - accuracy: 0.7345 - val_loss: 5658572.5000 - val_accuracy: 0.5000
Epoch 3/20
  1/326 [..............................] - ETA: 1:39 - loss: 4745132.0000 - accuracy: 0.5000
Epoch 00003: val_loss improved from 5658572.50000 to 4745132.00000, saving model to Bayesyian_CNN_binary.hdf5
326/326 [==============================] - 112s 344ms/step - loss: 5201420

In [11]:
# evaluate the model
test_steps_per_epoch = np.math.ceil(test_generator.samples / test_generator.batch_size)
scores = model.evaluate_generator(test_generator)
predictions = model.predict_generator(test_generator, steps=test_steps_per_epoch)
# Get most likely class
predicted_classes = np.argmax(predictions, axis=1)

true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())  


report = metrics.classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report) 


Instructions for updating:
Please use Model.evaluate, which supports generators.
Instructions for updating:
Please use Model.predict, which supports generators.
              precision    recall  f1-score   support

      NORMAL       0.38      1.00      0.55       234
   PNEUMONIA       0.00      0.00      0.00       390

    accuracy                           0.38       624
   macro avg       0.19      0.50      0.27       624
weighted avg       0.14      0.38      0.20       624



C:\Users\Sebi\.conda\envs\keras-gpu\lib\site-packages\sklearn\metrics\_classification.py:1268: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
# evaluate the modelBayes
test_steps_per_epoch = np.math.ceil(test_generator.samples / test_generator.batch_size)
scores = modelBayes.evaluate_generator(test_generator)
predictions = modelBayes.predict_generator(test_generator, steps=test_steps_per_epoch)
# Get most likely class
predicted_classes = np.argmax(predictions, axis=1)

true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())  


report = metrics.classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report) 

              precision    recall  f1-score   support

      NORMAL       0.38      1.00      0.55       234
   PNEUMONIA       0.00      0.00      0.00       390

    accuracy                           0.38       624
   macro avg       0.19      0.50      0.27       624
weighted avg       0.14      0.38      0.20       624

